In [1]:
import ndmg.utils as ndu
import ndmg.register as ndr
import ndmg.track as ndt
import ndmg.graph as ndg
import matplotlib.pylab as plt
import numpy as np
import nibabel as nb

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
"""
Create files list (will ordinarily be parsed from commandline)
"""
bval = '/Users/gkiar/Desktop/KKI2009_113_1_DTI.bval'
bvec = '/Users/gkiar/Desktop/KKI2009_113_1_DTI.bvec'
dti = '/Users/gkiar/Desktop/KKI2009_113_1_DTI.nii'
mprage = '/Users/gkiar/Desktop/KKI2009_113_1_MPRAGE.nii'
atlas = '/Users/gkiar/Desktop/MNI152_T1_1mm.nii.gz'
labels= '/Users/gkiar/Desktop/desikan.nii.gz'
mask = '/Users/gkiar/Desktop/MNI152_T1_1mm_brain_mask.nii.gz'
aligned_dti = '/Users/gkiar/Desktop/KKI2009_113_1_DTI_aligned.nii.gz'
temp_dti ='/tmp/dtifile.nii'

In [3]:
gtab = ndu().load_bval_bvec(bval, bvec, dti, temp_dti)

B-values shape (33,)
         min 0.000000 
         max 700.000000 
B-vectors shape (33, 3)
         min -0.996763 
         max 1.000000 
None


In [4]:
ndr().dti2atlas(temp_dti, gtab, mprage, atlas, aligned_dti)

Executing: flirt -in /tmp/dtifile_b0.nii.gz -ref /Users/gkiar/Desktop/KKI2009_113_1_MPRAGE.nii -omat /tmp/dtifile_KKI2009_113_1_MPRAGE_xfm.mat -cost mutualinfo -bins 256 -dof 12 -searchrx -180 180 -searchry -180 180 -searchrz -180 180
('', '')
Executing: flirt -in /Users/gkiar/Desktop/KKI2009_113_1_MPRAGE.nii -ref /Users/gkiar/Desktop/MNI152_T1_1mm.nii.gz -omat /tmp/KKI2009_113_1_MPRAGE_MNI152_T1_1mm_xfm.mat -cost mutualinfo -bins 256 -dof 12 -searchrx -180 180 -searchry -180 180 -searchrz -180 180
('', '')
Executing: flirt -in /tmp/dtifile.nii -ref /Users/gkiar/Desktop/MNI152_T1_1mm.nii.gz -out /Users/gkiar/Desktop/KKI2009_113_1_DTI_aligned.nii.gz -init /tmp/dtifile_MNI152_T1_1mm_xfm.mat -interp trilinear -applyxfm
('', '')


In [5]:
fibers = ndt().eudx_basic(aligned_dti, mask, gtab, seed_num=1000000)

In [6]:
labels_im = nb.load(labels)

graph = ndg(len(np.unique(labels_im.get_data()))-1, labels)
graph.make_graph(fibers)
g1 = graph.get_graph()
graph.summary()

# of Streamlines: 199007
0
25000
50000
75000
100000
125000
150000
175000
Graph attributes: None
Number of nodes: 70
Node attributes: ids
Number of edges: 1154
Edge attributes: weight


In [10]:

# plotting doesn't work atm

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.set_aspect('equal')
G1 = g1.get_adjacency(attribute='weight')
G1
#G2 = G1 + np.transpose(G1)

m = [[0.0, 1.47, 2.43, 3.44, 1.08, 2.83, 1.08, 2.13, 2.11, 3.7], [1.47, 0.0, 1.5,     2.39, 2.11, 2.4, 2.11, 1.1, 1.1, 3.21], [2.43, 1.5, 0.0, 1.22, 2.69, 1.33, 3.39, 2.15, 2.12, 1.87], [3.44, 2.39, 1.22, 0.0, 3.45, 2.22, 4.34, 2.54, 3.04, 2.28], [1.08, 2.11, 2.69, 3.45, 0.0, 3.13, 1.76, 2.46, 3.02, 3.85], [2.83, 2.4, 1.33, 2.22, 3.13, 0.0, 3.83, 3.32, 2.73, 0.95], [1.08, 2.11, 3.39, 4.34, 1.76, 3.83, 0.0, 2.47, 2.44, 4.74], [2.13, 1.1, 2.15, 2.54, 2.46, 3.32, 2.47, 0.0, 1.78, 4.01], [2.11, 1.1, 2.12, 3.04, 3.02, 2.73, 2.44, 1.78, 0.0, 3.57], [3.7, 3.21, 1.87, 2.28, 3.85, 0.95, 4.74, 4.01, 3.57, 0.0]]
matrix = np.matrix(m)

plt.imshow(matrix, interpolation='nearest', cmap=plt.cm.ocean)
plt.colorbar()
#plt.savefig(subLabel+'.png')
plt.show()

In [29]:
ggg = np.reshape(G1[2], [35,2])
plt.imshow(ggg, interpolation='nearest', cmap=plt.cm.ocean)
plt.colorbar()
#plt.savefig(subLabel+'.png')
plt.show()

In [ ]:
graph.save_graph()

In [ ]:
'eddy_correct ' + DTI_in + ' ' + DTI_out + ' 0'